In [ ]:
# ER iid
# $n$: Fixed as 1,024
# $p$: Fixed as 0.01
# $g$: 0 to 0.1, step size 0.01
# $R$: Fixed as 100,000 (as in our main experiments)

import numpy as np
from pathlib import Path
from decimal import Decimal, getcontext

getcontext().prec = 50

n = 1024
p = 0.01
p_string = str(p).replace(".", "p")
R = 100000
g_list = np.arange(0, 0.11, 0.01)
g_string_list = ["0p0", "0p01", "0p02", "0p03", "0p04", "0p05", "0p06", "0p07", "0p08", "0p09", "0p1"]

n_dec = Decimal("1024")
p_dec = Decimal("0.01")
R_dec = Decimal("100000")

p_save = Path("ER_iid")
p_save.mkdir(exist_ok=True)

for g, g_string in zip(g_list, g_string_list):
    g_dec = Decimal(str(g))
    try:
        p_round = (1 - (1 - p_dec) ** (1 / R_dec)) / (g_dec * g_dec)
    except:
        p_round = 0
            
    with open(p_save / f"n{n}_p{p_string}_g{g_string}_R{R}.txt", "w+") as f:
        # read the input file: number of nodes, density, binding strength, number of rounds
        f.write(f"{n} {p} {g} {R} {p_round}")


In [ ]:
# ER iter
# $n$: Fixed as 1,024
# $p$: Fixed as 0.01
# $g$: 0 to 0.1, step size 0.01
# $R$: Fixed as 100,000 (as in our main experiments)

import numpy as np
from pathlib import Path
from decimal import Decimal, getcontext

getcontext().prec = 50

n = 1024
p = 0.01
p_string = str(p).replace(".", "p")
R = 100000
g_list = np.arange(0, 0.11, 0.01)
g_string_list = ["0p0", "0p01", "0p02", "0p03", "0p04", "0p05", "0p06", "0p07", "0p08", "0p09", "0p1"]

n_dec = Decimal("1024")
p_dec = Decimal("0.01")
R_dec = Decimal("100000")

p_save = Path("ER_iter")
p_save.mkdir(exist_ok=True)

for g, g_string in zip(g_list, g_string_list):
    with open(p_save / f"n{n}_p{p_string}_g{g_string}_R{R}.txt", "w+") as f:
        # read the input file: number of nodes, density, binding strength, number of rounds
        f.write(f"{n} {p} {g} {R}")


In [ ]:
# CL
# $n$: Fixed as 1,024
# $p$: generating a power-law degree sequence $d_v$
# - overall density 0.01 and the power-law exponent is -2
# $g$: using the formula $g(d) = \gamma d^\alpha$ with 
# mean $g$: 0 to 0.1, step size 0.01
# \alpha: -1 (larger degree smaller $g$), 0 (same for all the nodes), 1 (larger degree larger $g$)
# $R$: Fixed as 100,000 as in our main experiments

from collections import defaultdict
from itertools import product
import numpy as np
from pathlib import Path
from networkx.algorithms.graphical import is_graphical
from networkx.utils.random_sequence import powerlaw_sequence
import pickle as pkl

n = 1_024
p = 0.01
R = 100_000
g_list = np.arange(0, 0.11, 0.01)
alpha_list = [-0.3, 0, 0.3]
t = 2

p_save = Path("CL")
p_save.mkdir(exist_ok=True)

if False:
    # find a valid degree sequence
    while True:  # Continue generating sequences until one of them is graphical
        seq = sorted([int(round(d)) for d in powerlaw_sequence(n, t)], reverse=True)  # Round to nearest integer to obtain DISCRETE degree sequence
        if is_graphical(seq):
            n_edges = sum(seq) / 2
            p_gen = n_edges / (n * (n - 1) / 2)
            print(p_gen)
            if abs(p_gen - p) < 0.001:
                break

    with open(p_save / "deg_seq.txt", "w+") as f:
        for deg in seq:
            f.write(f"{deg}\n")
            
    with open(p_save / "deg_seq.pkl", "wb") as f:
        pkl.dump(seq, f)

with open(p_save / "deg_seq.pkl", "rb") as f:
    deg_seq = pkl.load(f)

for g, alpha in product(g_list, alpha_list):
    weight_seq = [d ** alpha for d in deg_seq]
    target_g_sum = g * len(deg_seq)
    weight_seq = [w / sum(weight_seq) * target_g_sum for w in weight_seq]
    
    assert max(weight_seq) <= 1, f"{max(weight_seq)}"
    
    g_string = str(g).replace(".", "p")
    alpha_string = str(alpha).replace(".", "p")
    
    # read the input file
    # first line: n_round
    # each line after that: degree, number of nodes with this degree, alpha of this degree
    
    deg2num = defaultdict(int)
    deg2alpha = dict()
    
    for deg, w in zip(deg_seq, weight_seq):
        deg2num[deg] += 1
        deg2alpha[deg] = w
            
    with open(p_save / f"n{n}_p{p}_g{g_string}_alpha{alpha_string}_R{R}.txt", "w+") as f:
        f.write(f"{R}\n")
        for deg, num in deg2num.items():
            f.write(f"{deg} {num} {deg2alpha[deg]}\n")


In [ ]:
# SBM
# $n$: Fixed as 1,024
# Number of communities: 10
# Community sizes: Generating a power-law size sequence with exponent -2
# $p$: The intra-community edge probability and inter-community edge probability --> overall density 0.01
# $g$: using the formula $g(v) = \gamma {s_v}^\alpha$ with 
# $s_v$ is the size of the community that contains $v$
# mean $g$: 0 to 0.1, step size 0.01
# \alpha: -1 (larger community size smaller $g$), 0 (same for all the nodes), 1 (larger community size larger $g$)
# $R$: Fixed as 100,000 as in our main experiments

from collections import defaultdict
from itertools import product
import numpy as np
from pathlib import Path
from networkx.algorithms.graphical import is_graphical
from networkx.utils.random_sequence import powerlaw_sequence
import pickle as pkl

n = 1_024
n_comm = 10
p = 0.01
R = 100_000
g_list = np.arange(0, 0.11, 0.01)
alpha_list = [-0.5, 0, 0.5]
t = 1.5

p_save = Path("SBM")
p_save.mkdir(exist_ok=True)

if False:
    while True:
        # generate community size sequence
        seq = sorted([int(round(d)) for d in powerlaw_sequence(n_comm, t)], reverse=True)
        if min(seq) <= 2:
            continue
        if abs((sum(seq) / n) - 1) < 0.01:
            # adjust the size of the largest community to fit the total number of nodes
            max_idx = seq.index(max(seq))
            seq[max_idx] -= sum(seq) - n
            break

    with open(p_save / "comm_size_seq.pkl", "wb") as f:
        pkl.dump(seq, f)

# [545, 337, 49, 39, 16, 13, 9, 9, 4, 3]

with open(p_save / "comm_size_seq.pkl", "rb") as f:
    comm_seq = pkl.load(f)

n_pair_intra = sum([s * (s - 1) / 2 for s in comm_seq])
n_pair_inter = n * (n - 1) / 2 - n_pair_intra
target_edges = n * (n - 1) / 2 * p

p_intra = 0.02
edges_intra = n_pair_intra * p_intra

edges_inter = target_edges - edges_intra
p_inter = edges_inter / n_pair_inter

# print(p_intra, p_inter)

for g, alpha in product(g_list, alpha_list):
    weight_seq = [s ** alpha for s in comm_seq]
    sum_weight_raw = sum(w * s for w, s in zip(weight_seq, comm_seq))
    target_g_sum = g * n
    weight_seq = [w / sum_weight_raw * target_g_sum for w in weight_seq]
    assert max(weight_seq) <= 1, f"{max(weight_seq)}"
    
    g_string = str(g).replace(".", "p")
    alpha_string = str(alpha).replace(".", "p")
    
    with open(p_save / f"n{n}_p{p}_g{g_string}_alpha{alpha_string}_R{R}.txt", "w+") as f:
        # first line: n_blocks, n_round    
        f.write(f"{n_comm} {R}\n")
        # after that, the n_blocks lines are the binding strength of each block
        for s, w in zip(comm_seq, weight_seq):
            f.write(f"{w}\n")
        # after that, the n_blocks lines are the probabilities of each block
        for i, s in enumerate(comm_seq):
            for j, s2 in enumerate(comm_seq):
                if i == j:
                    f.write(f"{p_intra}")
                else:
                    f.write(f"{p_inter}")
                if j != len(comm_seq) - 1:
                    f.write(" ")
            f.write("\n")
        # finally, the n_blocks lines are the number of nodes in each block
        for s in comm_seq:
            f.write(f"{s}\n")
    

In [ ]:
# Kronecker
# $n$: Fixed as 1,024 (Seed matrix $2 \times 2$, Kronecker power 10)
# $p$: Seed matrix [0.95, 0.63; 0.63, 0.32]
# $g$: using the formula $g(v) = \gamma (i_v + 1)^\alpha$ with 
# $i_v$ is the number of ones in the binary node labels of $v$
# mean $g$: 0 to 0.1, step size 0.01
# \alpha: -1 (more ones smaller $g$), 0 (same for all the nodes), 1 (fewer ones larger $g$)

from collections import defaultdict
from itertools import product
import math
import numpy as np
from pathlib import Path
from networkx.algorithms.graphical import is_graphical
from networkx.utils.random_sequence import powerlaw_sequence
import pickle as pkl

n = 1_024
size_seed = 2
k = 10  # kronecker power
p = 0.01
R = 100_000
g_list = np.arange(0, 0.11, 0.01)
alpha_list = [-1, 0, 1]
t = 1.5

p_save = Path("KR")
p_save.mkdir(exist_ok=True)

seed_matrix = np.array([[0.95, 0.63], [0.63, 0.32]])

numOfOne2numNodes = dict()
for i in range(k + 1):
    # the number of length-k binary strings with i ones
    numOfOne2numNodes[i] = math.comb(k, i)

for g, alpha in product(g_list, alpha_list):
    # first line: k, n_rounds, size of seed matrix
    # (k+1) lines after that: each line is the alpha for nodes with that number of zeros in their binary representation
    # the last lines: the seed matrix
    
    g_string = str(g).replace(".", "p")
    alpha_string = str(alpha).replace(".", "p")
    
    weight_seq = [(i + 1) ** alpha for i in range(k + 1)]
    sum_weight_raw = sum(w * numOfOne2numNodes[i] for i, w in enumerate(weight_seq))
    target_g_sum = g * n
    weight_seq = [w / sum_weight_raw * target_g_sum for w in weight_seq]
    assert max(weight_seq) <= 1, f"{max(weight_seq)}"
    
    with open(p_save / f"n{n}_p{p}_g{g_string}_alpha{alpha_string}_R{R}.txt", "w+") as f:
        f.write(f"{k} {R} {size_seed}\n")
        for w in weight_seq:
            f.write(f"{w}\n")
        for row in seed_matrix:
            for i_col, val in enumerate(row):
                f.write(f"{val}")
                if i_col != len(row) - 1:
                    f.write(" ")
            f.write("\n")
    